In [2]:
import networkx as nx
import pandas as pd
import numpy as np
import matplotlib as plt
from tqdm import tqdm, trange
import torch
import pickle

In [ ]:
#set parameters
vertices = 1000
adjacencies = 100
pi_vals = [0.25, 0.5, 0.75]
# pi_vals = [0.25]
num_trials = 1000

In [ ]:
replicate = {}

In [ ]:
device = torch.device("cuda")

In [ ]:
# c is treatment assignment
# beta 1 is treatment assignment
# beta 2 is first degree neighbors
# beta 3 is second degree neighbors
def add_test_to_dictionary(beta_1, beta_2, beta_3, replicate):
  for pi in pi_vals:
    ADE = []
    AIE_1 = []
    AIE_2 = []
    AIE_3 = []
    AIE_4 = []
    for i in trange(1000):
      G = nx.random_regular_graph(adjacencies, vertices)

      # first degree neighbor matrix
      A = nx.adjacency_matrix(G).todense()

      # second degree neighbor matrix
      A_gpu = torch.tensor(A).float().to(device)
      A_2 = torch.matmul(A_gpu, A_gpu).cpu().numpy()
      A_2 = np.where(A_2 > 0, 1, 0)
      np.fill_diagonal(A_2, 0)
      A_2 = A_2 - A # second degree neighbors
      A_2_gpu = torch.tensor(A_2).float().to(device)

      # treatment assignment
      W = np.random.binomial(1, pi, vertices)
      W_gpu = torch.tensor(W).float().to(device)

      # noise
      noise = np.random.normal(0, 1, vertices)

      # generate Y
      # no interference
      if beta_2 == 0 and beta_3 == 0:
        ei = beta_1
      # only first degree neighbor
      elif beta_3 == 0:
        ei = beta_1 + beta_2 * torch.matmul(W_gpu, A_gpu).cpu().numpy() / adjacencies
      # only second degree neighbor
      elif beta_2 == 0:
        ei = beta_1 + beta_3 * torch.matmul(W_gpu, A_2_gpu).cpu().numpy() / np.sum(A_2, axis=1)
      # first and second degree neighbor
      else:
        ei = beta_1 + beta_2 * (torch.matmul(W_gpu, A_gpu).cpu().numpy() / adjacencies) + beta_3 * (torch.matmul(W_gpu, A_2_gpu).cpu().numpy() / np.sum(A_2, axis=1))

      Y = W * (ei - 3 * (ei - 1/2)**3) + noise

      # ADE
      average_direct_effect = ((np.dot(Y,W))/(pi) - (np.dot(Y, (1-W)))/(1-pi))/vertices
      ADE.append(average_direct_effect)

      # AIE
      # E = first degree
      average_indirect_effect1 = np.dot(Y, ((W @ A)/pi - ((1 - W) @ A) / (1 - pi))) / vertices
      average_indirect_effect2 = np.dot(Y, ((W @ A_2)/pi - ((1 - W) @ A_2) / (1 - pi))) / vertices
      AIE_1.append(average_indirect_effect1)
      AIE_2.append(average_indirect_effect2)

    str1 = str(pi)+" ADE_{}_{}_{}".format(round(beta_1, 2), round(beta_2, 2), round(beta_3, 2))
    str2 = str(pi)+" AIE_1_{}_{}_{}".format(round(beta_1, 2), round(beta_2, 2), round(beta_3, 2))
    str3 = str(pi)+" AIE_2_{}_{}_{}".format(round(beta_1, 2), round(beta_2, 2), round(beta_3, 2))
    replicate[str1] = ADE
    replicate[str2] = AIE_1
    replicate[str3] = AIE_2

    with open('12_9_23_Equation_3_RandomGraph.pkl', 'wb') as fp:
      pickle.dump(replicate, fp)


In [ ]:
# betas = []
# # baseline where no interference
# betas.append((0, 0))

# # only first degree neighbors
# betas.append((1, 0))

# # first and second degree neighbors
# betas.append((1/3, 2/3))
# betas.append((1/2, 1/2))
# betas.append((2/3, 1/3))
betas = []
# baseline where no interference
betas.append((1, 0, 0))

#only first degree neighbors
betas.append((1/3, 2/3, 0))
betas.append((0.5, 0.5, 0))
betas.append((2/3, 1/3, 0))

#only second degree neighbors
betas.append((1/3, 0, 2/3))
betas.append((0.5, 0, 0.5))
betas.append((2/3, 0, 1/3))

#first and second degree neighbors
betas.append((1/3, 5/9, 1/9))
betas.append((1/3, 1/3, 1/3))
betas.append((1/3, 1/9, 5/9))

In [ ]:
# only second degree neighbors
# betas.append((0, 1))

In [ ]:
for beta in betas:
  add_test_to_dictionary(beta[0], beta[1], beta[2], replicate)

100%|██████████| 1000/1000 [06:06<00:00,  2.73it/s]


In [1]:
df_r = pd.DataFrame.from_dict(replicate)
df_r.head()

NameError: ignored

In [ ]:
df_r.mean()

In [ ]:
df_r.to_csv('12_9_23_Equation_3_RandomGraph.csv')